## CONVERSOR DE ARQUIVOS DE IMAGEM RAW DA MISSAO CASSINI - NASA/JPL/PDS
#### Wandeclayt M./Projeto Ceu profundo *@ceuprofundo*

LGPL 2.0

Este script importa arquivos de imagem da missão Cassini-Huygens e de outras missões, 
disponibilizados no Planetary Data Service, busca informações sobre o formato dos dados
no arquivo \*.LBL e exporta um novo arquivo, compatível com editores de imagem.

In [ ]:
#Conversor de Arquivos de Imagem RAW (IMG)
#do Planetary Data System (PDS) do JPL
#
#Author: Wandeclayt M./@CeuProfundo
#
# 1. Importa arquivo como Array Numpy 1D;
# 2. Remove os Bytes iniciais (Header + Telemetria) do Array;
# 3. Altera (Reshape) a geometria do Array para 2D (linhas, colunas)
#
# N1836255843_1.RAW


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import misc
from scipy import ndimage
import math
import re

Insira aqui o nome do arquivo LBL associado ao arquivo de imagem.

In [ ]:
#Importa "header" (Arquivo *.LBL)

LBL = input("Arquivo LBL :")
with open(LBL) as Header:
    arquivoHeader = Header.read()
print(arquivoHeader)

In [ ]:
#Procura no texto do header (LBL) usando RegEx:
#o tamanho em Bytes e o nome do arquivo.
#os bytes de header e telemetria
#o formato da imagem
#a profundidade de bits


recordBytes = re.findall(r"^RECORD_BYTES\s=\s(.*)$",arquivoHeader,re.M)
recordBytes = int(recordBytes[0])
print("recordBytes = " + str(recordBytes))

#Image Name

imageName = re.findall(r"IMAGE\s=\s\(\"(.*)\"",arquivoHeader,re.M)
imageName = str(imageName[0])
print("imageName = " + imageName)

#Discard Bytes

discardBytes = re.findall(r"\sBYTES\s=\s(.*)$",arquivoHeader,re.M)
telemetryStartByte = re.findall(r"START_BYTE\s=\s(.*)$", arquivoHeader, re.M)

########   Header

headerBytes = int(discardBytes[0]) 

########   Telemetry

telemetryBytes = int(discardBytes[1])
telemetryStartByte = int(telemetryStartByte[0])

#Sample Bits

sampleBits = re.findall(r"SAMPLE_BITS\s=\s(.*)$",arquivoHeader,re.M)
sampleBytes = int(int(sampleBits[0])/8)
print("sampleBytes = " + str(sampleBytes))

########   Discard

discardBytes = (headerBytes + telemetryBytes + telemetryStartByte)
discardSamples = int(discardBytes/sampleBytes)
print("headerBytes = " + str(headerBytes))
print("telemetryBytes = " + str(telemetryBytes))
print("telemetryStartByte = " + str(telemetryStartByte))
print("discardBytes = " + str(discardBytes) + " (Header + Telemetry Bytes + telemetryStartByte)")
print("discardSamples = " + str(discardSamples) + " (discardBytes/discardSamples)")

#Image Lines

imageLines = re.findall(r"\sLINES\s=\s(.*)$",arquivoHeader,re.M)
imageLines = int(imageLines[0])
print("imageLines = " + str(imageLines))

#Line Samples

lineSamples = re.findall(r"\sLINE_SAMPLES\s=\s(.*)$",arquivoHeader,re.M)
lineSamples = int(lineSamples[0])
print("lineSamples = " + str(lineSamples))

#Prefix Bytes

prefixBytes = re.findall(r"\sLINE_PREFIX_BYTES\s=\s(.*)$",arquivoHeader,re.M)
prefixBytes = int(prefixBytes[0])
print("prefixBytes = " + str(prefixBytes))

extendedLine = int(lineSamples + prefixBytes/sampleBytes)
print("extendedLine = " + str(extendedLine) + " (lineSamples + prefixBytes/sampleBytes)")


In [ ]:
datatype = 'int' + str(sampleBytes * 8)
datatype

In [ ]:
ArquivoRaw = np.fromfile(imageName, dtype=datatype, sep="")
print("ArquivoRaw.shape: %d", ArquivoRaw.shape)
print("ArquivoRaw: ", ArquivoRaw)

shape = ArquivoRaw.shape[0]
print(shape)

print(str(1024 * 1036))

ArquivoSemHeader = ArquivoRaw[discardSamples:]
print(ArquivoSemHeader.shape)

Imagem = ArquivoSemHeader.reshape(imageLines, extendedLine)
print(ArquivoSemHeader.shape)

In [ ]:
with open('Teste16.RAW', 'wb') as f:
    np.save(f, Imagem)

In [ ]:
plt.imshow(Imagem, cmap="gray")
plt.savefig("PDS.png")
plt.show()

In [ ]:
#NomeDoArquivo = imageName
#BytesHeader = headerBytes
#BytesTelemetria = telemetryBytes
#ByteInicialdaImagem = discardBytes
#sampleBits = int(input("sample bits: "))
#ImageLines = int(input("image lines: "))
#ImageCols = int(input("image cols: "))
#PrefixBytes = int(input("prefix bytes: "))
#PrefixPixels = int(8*PrefixBytes / SampleBits)